### **This Notebook Performs a Parameters Search on Every Possible Cleaning and Model Parameter.**

Lets test different spectral cleaning parameters by building a model pipeline.

Import Libraries

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, GroupKFold, cross_val_score
from Spectra_Preparation_Functions import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import optuna

Read the spectral data

In [3]:
#df = pd.read_csv("../../data/exosomes.raw_spectrum_1.csv")
# df = pd.read_csv("../../data/raw_df_outliers_removed.csv")
df = pd.read_csv("../../data/400-1800_with_raw_scaled_surface_pagerank.csv")

In [4]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status,PageRank
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal,0.610024
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal,0.610024
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal,0.610024
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal,0.610024
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal,0.610024
...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia,1.201164
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia,1.201164
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia,1.201164
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia,1.201164


In [5]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')
wavelength_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1765.6628,1774.7809,1769.0302,1756.4220,1758.8690,1763.2300,1745.2285,1773.3534,1774.7166,1753.3281,...,1210.4993,1213.9619,1225.2153,1210.0010,1210.6858,1194.4679,1195.1451,1189.8683,201210-1,Normal
201210-1-01,1966.9930,1962.4237,1954.5616,1954.3228,1963.0917,1975.0807,1979.3162,1963.4561,1968.4587,1964.0000,...,1382.6973,1363.7004,1360.6210,1354.0477,1353.0381,1353.9978,1361.2426,1370.2874,201210-1,Normal
201210-1-02,2182.6694,2149.6565,2146.0227,2159.3459,2167.2910,2160.9861,2145.6575,2134.2004,2142.8303,2138.6309,...,1976.2070,1989.0183,1996.2838,1979.3507,1976.2002,1994.9839,1974.2030,1971.1880,201210-1,Normal
201210-1-03,2445.0837,2430.4973,2422.7927,2434.3433,2454.9700,2462.8245,2454.7007,2467.7329,2449.5161,2421.3474,...,1992.3817,2022.6331,2001.8311,2010.0946,2006.4933,2017.2891,2038.1699,2000.6475,201210-1,Normal
201210-1-04,2250.4536,2248.6235,2245.0984,2242.7173,2235.2803,2228.9585,2236.0095,2229.6091,2225.9231,2211.0359,...,2009.0385,1953.3303,1963.5698,1964.5299,1969.5634,1986.6266,1970.1484,2007.0848,201210-1,Normal


>#### **Use Optuna to Find the Best Cleaning Parameter**

In [6]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status,PageRank
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal,0.610024
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal,0.610024
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal,0.610024
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal,0.610024
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal,0.610024
...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia,1.201164
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia,1.201164
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia,1.201164
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia,1.201164


In [7]:
def objective(trial):

    # Decide which cleaning steps to implement
    pagerank_cutoff = trial.suggest_float('pagerank_cutoff', 0, 1)
    despike = trial.suggest_categorical('despike', [True, False])
    baseline_correct = trial.suggest_categorical('baseline_correct', [True, False])
    smoothing = trial.suggest_categorical('smoothing', [True, False])
    scaling = trial.suggest_categorical('scaling', [True, False])

    df_copy = df.copy()

    # Filter less central spectra in each surface
    df_copy = df_copy[df_copy['PageRank'] > pagerank_cutoff]

    # Apply preprocessing based on suggested parameters
    if despike:
        despike_ma = trial.suggest_int('despike_ma', 5, 100)
        despike_threshold = trial.suggest_float('despike_threshold', 3, 10, step=0.25)
        df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: despike_group(x, ma=despike_ma, threshold = despike_threshold))

    if baseline_correct:

        # Trial a different number of baseline correction
        baseline_correction_type = trial.suggest_categorical('baseline_correction_type', ['asls', 'iasls', 'airpls', 'poly', 'psalsa'])
        #baseline_correction_type = trial.suggest_categorical('baseline_correction_type', ['asls', 'iasls', 'airpls', 'poly'])
        if baseline_correction_type == 'asls': 
            lam = trial.suggest_categorical('asls_lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])
            p = trial.suggest_float('asls_p', 0.001, 0.1, step=0.001)
            df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam=lam, p=p))
            df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']
        
        elif baseline_correction_type == 'iasls':
            lam = trial.suggest_categorical('iasls_lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])
            p = trial.suggest_float('iasls_p', 0.001, 0.1, step=0.001)
            lam_1 = trial.suggest_categorical('iasls_lam_1', [10**-3, 10**-4, 10**-5])
            df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: iasls_baseline_correction(x, lam=lam, p=p, lam_1=lam_1))
            df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']

        elif baseline_correction_type == 'airpls':
            lam = trial.suggest_categorical('airpls_lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])
            df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: airpls_baseline_correction(x, lam=lam))
            df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']

        elif baseline_correction_type == 'poly':
        #else:
            poly_order = trial.suggest_int('polyfit_poly_order', 1, 5)
            df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: poly_baseline_correction(x, poly_order=poly_order))
            df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']

        else:
            lam = trial.suggest_categorical('psalsa_lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])
            p = trial.suggest_float('psalsa_p', 0.1, 0.9, step=0.05)
            df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: psalsa_baseline_correction(x, lam=lam, p=p))
            df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']

    if smoothing:
        window_size = trial.suggest_int('window_size', 5, 201)
        poly_order = trial.suggest_int('smoothing_poly_order', 1, 4)
        df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: savgol_filter(x, window_size, poly_order, deriv=0))

    if scaling:
        scaling_type = trial.suggest_categorical('scaling_type', ['normal', 'vector', 'svn'])
        if scaling_type == 'normal':
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: normalise(x))
        elif scaling_type == 'vector':
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: vector_normalise(x))
        else:
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: svn_normalise(x))

    # Prepare data for ML
    wavelength_df = prepare_wavelength_df(df_copy, 'Absorbance')
    X = wavelength_df.drop(['Status', 'SurID'], axis=1)
    y = wavelength_df['Status']
    groups = wavelength_df['SurID']

    # Suggest classifier type
    classifier_name = trial.suggest_categorical("classifier", ["ExtraTrees", "RandomForest", "SVC"])

    if classifier_name == "ExtraTrees":
        classifier = ExtraTreesClassifier(
            n_estimators=trial.suggest_int("et_n_estimators", 10, 200),
            max_depth=trial.suggest_int("et_max_depth", 2, 32, log=True),
            criterion=trial.suggest_categorical("et_criterion", ["gini", "entropy", "log_loss"]),
            random_state=1234,
        )

    elif classifier_name == "RandomForest":
        classifier = RandomForestClassifier(
            n_estimators=trial.suggest_int("rf_n_estimators", 10, 200),
            max_depth=trial.suggest_int("rf_max_depth", 2, 32, log=True),
            criterion=trial.suggest_categorical("rf_criterion", ["gini", "entropy", "log_loss"]),
            random_state=1234,
        )
        
    else: # SVC
        classifier = SVC(
            C=trial.suggest_float("svc_c", 1e-10, 1e10, log=True),
            kernel=trial.suggest_categorical("svc_kernel", ["linear", "poly", "rbf", "sigmoid"]),
            gamma=trial.suggest_categorical("svc_gamma", ["scale", "auto"]),
            random_state=1234,
        )

    cv = GroupKFold(n_splits=10)

    # Perform cross-validation
    scores = cross_validate(classifier, X, y, groups=groups, cv=cv, scoring='accuracy')

    # Return the average accuracy across all folds
    return np.mean(scores['test_score'])


# sampler = TPESampler(seed=10)  # Make the sampler behave in a deterministic way.
# study = optuna.create_study(direction='maximise', sampler=sampler)

study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100, n_jobs=-1)
study.optimize(objective, timeout=100, n_jobs=-1)

print(study.best_trial)

[I 2024-04-02 00:51:32,099] A new study created in memory with name: no-name-7670a54d-27c7-4b6a-b864-7b5b260ed461
[I 2024-04-02 00:52:17,611] Trial 6 finished with value: 0.6307570110013414 and parameters: {'pagerank_cutoff': 0.8708442746749662, 'despike': False, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'scaling_type': 'vector', 'classifier': 'SVC', 'svc_c': 454.21397912957974, 'svc_kernel': 'rbf', 'svc_gamma': 'scale'}. Best is trial 6 with value: 0.6307570110013414.
[I 2024-04-02 00:52:20,298] Trial 11 finished with value: 0.2702022977435311 and parameters: {'pagerank_cutoff': 0.9150758382449748, 'despike': False, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'scaling_type': 'normal', 'classifier': 'SVC', 'svc_c': 2.9695880580466346e-05, 'svc_kernel': 'rbf', 'svc_gamma': 'auto'}. Best is trial 6 with value: 0.6307570110013414.
[I 2024-04-02 00:52:30,437] Trial 10 finished with value: 0.48415350441263527 and parameters: {'pagerank_cutoff': 0.38

In [ ]:
results_df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
results_df.to_csv("../../data/studies/all_models_extra_baseline+outliers.csv")

In [ ]:
import joblib

joblib.dump(study, "../../data/studies/all_models_extra_baseline+outliers.pkl")

['../../data/studies/all_models_extra_baseline+outliers.pkl']

In [ ]:
# loaded_study = joblib.load("../../data/studies/cleaning_study.pkl")
# print("Best trial until now:")
# print(" Value: ", loaded_study.best_trial.value)
# print(" Params: ")
# for key, value in loaded_study.best_trial.params.items():
#     print(f"    {key}: {value}")

In [ ]:
print(study.best_trial)

FrozenTrial(number=21, state=1, values=[0.6694482081004609], datetime_start=datetime.datetime(2024, 3, 31, 23, 5, 6, 115062), datetime_complete=datetime.datetime(2024, 3, 31, 23, 9, 31, 711423), params={'pagerank_cutoff': 1.0068762446845378, 'despike': True, 'baseline_correct': True, 'smoothing': False, 'scaling': True, 'despike_ma': 30, 'despike_threshold': 3.25, 'baseline_correction_type': 'poly', 'polyfit_poly_order': 1, 'scaling_type': 'vector', 'classifier': 'RandomForest', 'rf_n_estimators': 198, 'rf_max_depth': 26, 'rf_criterion': 'entropy'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'pagerank_cutoff': FloatDistribution(high=1.5, log=False, low=0.0, step=None), 'despike': CategoricalDistribution(choices=(True, False)), 'baseline_correct': CategoricalDistribution(choices=(True, False)), 'smoothing': CategoricalDistribution(choices=(True, False)), 'scaling': CategoricalDistribution(choices=(True, False)), 'despike_ma': IntDistribution(high=100, log=Fal

In [ ]:
from optuna.visualization import plot_optimization_history, plot_slice

plot_optimization_history(study)

In [ ]:
plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

Continue the study.

In [ ]:
# study.optimize(objective, n_trials=250, n_jobs=-1)

# print(study.best_trial)

[I 2024-03-31 23:48:57,972] Trial 122 finished with value: 0.5383957221598592 and parameters: {'pagerank_cutoff': 1.137543141791174, 'despike': True, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'despike_ma': 74, 'despike_threshold': 3.0, 'scaling_type': 'svn', 'classifier': 'SVC', 'svc_c': 4199314.117954566, 'svc_kernel': 'rbf', 'svc_gamma': 'scale'}. Best is trial 21 with value: 0.6694482081004609.
[I 2024-03-31 23:48:58,773] Trial 120 finished with value: 0.5231878092858863 and parameters: {'pagerank_cutoff': 1.1342384260937064, 'despike': True, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'despike_ma': 79, 'despike_threshold': 3.0, 'scaling_type': 'svn', 'classifier': 'SVC', 'svc_c': 2001113.33371708, 'svc_kernel': 'rbf', 'svc_gamma': 'scale'}. Best is trial 21 with value: 0.6694482081004609.
[I 2024-03-31 23:49:23,341] Trial 116 finished with value: 0.6399181317961626 and parameters: {'pagerank_cutoff': 0.8646493208610616, 'despike': True, 'ba

ValueError: p must be between 0 and 1

In [ ]:
results_df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
results_df.to_csv("../../data/studies/all_models_extra_baseline+outliers.csv")

In [ ]:
import joblib

joblib.dump(study, "../../data/studies/all_models_extra_baseline+outliers.pkl")

['../../data/studies/all_models2.pkl']